# DMR Data Science Intern Takeaway Task

### Introduction

This notebook will walk you through a subset of the tasks required to build and maintain our models and do basic analysis. The final output will be modified version of this notebook which trains a basic model and displays a classification report. This files should be sent back to <a href="mailto:jsands@digital-mr.com?subject=Data%20Science%20Intern%20Task">jsands@digital-mr.com</a> no later than 1 week after receiving this archive. 

All code should be implemented in this notebook and feel free to add any additional comments/code blocks/figures as you see fit. You are not expected to have all the knowledge required to do complete the task immediately, but acquire it through a process of research and experimentation.

This particular model will curate publicly available text written in Japanese, do some basic analysis, then train a basic machine learning model that is capable of predicting sentiment (positive, negative or neutral) of an unseen piece of text written in Japanese.

Be sure to include explanations in markup or python comments as you go so that your work can easily be inspected and reviewed by others.

### Import Libraries

Below is a list of modules that are either necessary or that you may find useful. Feel free to expand upon these if required.

In [1]:
# Imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from nltk import TweetTokenizer

### Read Data

Inspect the data in the archive and read them into separate pandas dataframes. Do some basic curation then combine them into one. The first one has been done for you.

The wikipedia content is assumed to be neutral sentiment. It also disbalances the data set so only sample 10,000 documents.

The JRTE data was curated by multiple curators per text, each assigning either -1, 0 or 1 to the text which correspond with negative, neutral or positive, respectively. Handle this data as you see fit.

In [2]:
#--- Darkmap Amazon Review Dataset ---#
# Read positive text
df_amazon_jap_neg = pd.read_csv("./data/amazon-jap/10000negative.txt",
                                sep="\\n",
                                header=None,
                                names=["content"],
                                engine='python'
                               )

# Read negative text
df_amazon_jap_pos = pd.read_csv("./data/amazon-jap/10000positive.txt",
                                sep="\\n",
                                header=None,
                                names=["content"],
                                engine='python'
                               )

# Assign sentiment column to the correct text
df_amazon_jap_neg["sentiment"] = "negative"
df_amazon_jap_pos["sentiment"] = "positive"

# Combine the dataframes
df_amazon_jap = pd.concat([df_amazon_jap_neg,df_amazon_jap_pos],
                          sort=False
                         )

# Reset the index
df_amazon_jap.reset_index(drop=True, inplace=True)

# Create a source column to distinguish this data from the other data sources
df_amazon_jap["source"] = "Darkmap Amazon Review Dataset"

# Display option to stop content clipping
pd.set_option('display.max_colwidth', None)

# Display the sentiment distribution and the first 5 entries for manual inspection
print('--------------------- Amazon Japan Sentiment Distribution ---------------------')
print(df_amazon_jap.sentiment.value_counts())
print('-'*79)
df_amazon_jap.head()

--------------------- Amazon Japan Sentiment Distribution ---------------------
negative    10000
positive    10000
Name: sentiment, dtype: int64
-------------------------------------------------------------------------------


,content,sentiment,source
0,機能 制限 で アダルト 制限 して た のに 、 全然 機能 して ませ ん でした 。 我が家 の 中 坊 が 見 まくって いた 履歴 が あって 、 発覚 。 痕跡 を 残す ところ が 、 まだ 子供 です が 。 。 。 Kindle や 、 Android アプリ の 視聴 機能 制限 と は 連動 して ない そうです 。 カスタマー サポート に 確認 済み 。 Amazon が 対応 して くれる まで 、 我が家 で は 、 保護 者 が 家 を 空ける とき は 、 テレビ から スティック を 抜いて いく こと に なり そうです 。 すごく お 気に入り だった のに 、 そこ が かなり マイナスです 。 以下 、 カスタマー センター と の チャット の コピー です 。 お 問い合わせ 内容 ： Fire tv stick の 機能 制限 で 成人 向け コンテンツ の 制限 を オン に して も 、 R15 の ビデオ が 再生 さ れて しまい ます 。 Kindle や スマホ で は コンテンツ に カギ マーク が でて R15 の 制限 が 出来る のです が 、 fire tv で は それ が でき ませ ん 。 どう やって 設定 する のです か 。,negative,Darkmap Amazon Review Dataset
1,本日 この 商品 が 届いて 、 無事に セットアップ が 済み ました 。 家族 で テレビ の 前 に 座って Fire TV Stick を 起動 さ せる と 、 トップ ページ 内 の 「 お すすめ の 日本 映画 」 に 、 声 に する の も はばから れる タイトル 、 下着 姿 の 女性 など 一見 して アダルト 向け ビデオ の それ と わかる 商品 写真 が ずらり と 並び ます 。 電源 を 入れて 現れる トップ ページ です よ ？ もちろん 言う まで も なく 、 「 18 歳 以下 の 視聴 制限 」 は オン で 、 閲覧 でき ない ように なって い ます 。 サポート に 尋ねる と 、 「 18 歳 以下 が 再生 する こと は でき なく なる が 、 お すすめ や 検索 結果 に は 表示 さ れて しまう の が 仕様 である 」 と の こと でした 。 要は 、 この アダルト 向け ビデオ の 表示 を 消す こと は 出来 ない と いう こと です 。 家族 で みんな で 唖然と し ました 。 映画 を 視聴 する たび に 気まずい 空気 に なる なんて 。 ちなみに Apple TV は そんな 画面 は 出て き ませ ん 。 早速 返品 さ せて いただき ました 。 テレビ は タブレット 端末 や スマートフォン と 違い 、 老若 男女 、 みんな で 見る 機会 が 多い もの です 。 Amazon さん と して は 、 こういった こと に 細心の 注意 を 払う こと が 必要で は ない でしょう か 。 配慮 の 足ら な さ を 感じ ます 。,negative,Darkmap Amazon Review Dataset
2,2015/10/29 に 届き ました 。 妻 へ の 誕生日 プレゼント の 一 つ だった ので すぐ 開封 して 動作 確認 し ました 。 Amazon の オープニング 画面 の 後 ブラックアウト 。 何 分 待って も 何 回 試して も ダメだった ので 他の テレビ に 接続 しよう と 思い スティック に 手 を かける と 発熱 して 熱くて 触れ ませ ん でした 。 すぐ に 交換 処理 を しよう と Amazon に 接続 。 交換 の 手続き を しよう と 思い 進んで いく と カスタマー に 連絡 と いう 選択肢 しか 選べ なく なり 仕方なく カスタマー の M さん と チャット 。 何 か の Amazon 側 の トラブル で 交換 処理 が 出来 ない と の こと 。 とても 楽しみに して いた もの な ので きちんと 対応 を して いただける なら 待ち ます と M さん に 伝えた 。 しっかり 対応 する と M さん が 約束 して くれた ので 待つ こと に 。 それ から 丸 ３ 日 待った が 音沙汰 なし 。 Amazon に 不信 感 を 抱き 今度 は メール にて カスタマー に 連絡 。 当然 チャット を した M さん と は 別の 担当 K さん 。 経緯 を 伝えた が 前回 担当 の M さん が 調査 して メール する から 待て と の 旨 の 定型 文 が 返って きた 。 仕方なく もう １ 日 待って みた 。 当然 音沙汰 なし 。 どう なって いる の か ？ と 、 もう 一 度 メール を 送る と 今度 は Ｏ さん が 対応 して 前回 担当 の K さん が 調査 して メール する から 待て と の 旨 の 定型 文 が 送ら れて きた 。 しかし 、 Ｏ さん の 定型 文 に は 『 交換 対応 が 出来 ない かも しれ ない 』 旨 の 文章 が 付け加え られて おり ますます 不安に なった ので 、 交換 出来 ない 可能 が ある の か と 再度 メール にて 質問 する と 、 Ｏ さん から すぐ に 返信 が あり K さん が 調査 して る から 待て と の メール が 返って きた 。 ほんと は ここ に チャット から の やり取り を 全部 貼り付けよう か と 思った けど 、 気持ちよく Amazon を 使って 居ら れる 方 も 大勢 いらっしゃる と 思う ので やめ ました 。 機械 もの な ので 初期 不良 は 仕方 あり ませ ん が 、 Amazon カスタマー の 果たす べき 責任 を うやむやに した たらい回し に この先 も 付き合わ さ れる の か と 思う と 、 正直 嫌に なって き ました 。 私 の 望み は ただ ひと つ 。 良品 と 交換 して ください 。 2015/11/4 最初の 担当 者 の M さん から 交換 の 手続き が でき ました と の 旨 の メール が あり ました 。 最終 的に ちゃんと 対応 して もらえ ました が 、 M さん 以外 の 方 の 対応 に 誠意 が 感じ られ なかった ので 残念でした 。 2015/11/8 交換 商品 の 到着 予定 日 より 少し 早く 代わり の 商品 が 届き ました 。 少し 早く 発送 できる 旨 を カスタマー の U さん が わざわざ 電話 して くれ ました 。 更に 一連の 流れ に つき 丁寧に 謝罪 して くれ ました 。 はじめ から この 対応 だったら … と 、 ますます 残念に … やっと ここ で 商品 の レビュー な のです が 、 設定 は とにかく 簡単です 。 機械 の こと と か 全く 解ら なくて も 誰 に でも できる と 思い ます 。 ホーム 画面 も 思って いた より も 使い やすい のです が 、 音声 認識 機能 付 の リモコン もしくは スマホ アプリ を 使わ ない と 検索 機能 が 少し 使い にくい です 。 私 の 家 の 環境 で は Wi-Fi が 途切れる こと も なく 、 再生 も スムーズに 行わ れ ます 。 使い やす さ と コスト パフォーマンス を 考えたら 商品 自体 は 星 ５ つ です 。,negative,Darkmap Amazon Review Dataset
3,レビュー に つら れて 購入 。 早速 、 解説 どおり に TV に 接続 開始 、 ところが 全然 反応 なし ！ いろいろ 試して も だめだった ので 知恵 袋 を 参照 したら 、 我が家 の TV は 亀山 モデル の AQUOS ！ この 型 に は 反応 し ない と の 回答 を みつけた 次第 で 、 なに これ ～！ それ ぐらい の 注意書き ぐらい あって 然る べき か と ！,negative,Darkmap Amazon Review Dataset
4,設定 は デフォルト の まま です が 、 視聴 出来る 動画 の 数 が 、 PC 版 と 全く 違って 少ない です 。 ちょっと 話 に なら ない くらい 少ない です 。 残念 。,negative,Darkmap Amazon Review Dataset


In [ ]:
# Your code and/or markup here.

### Complete Preprocessor Class

Below is a python class which can be used to clean the data. Please study it and add a method to strip all text of the symbols contained within the list `SPEC_CHARS`.

In [1]:
# Your code and/or markup here wherever you see fit.
class TextPreProcessor():
    """
    Rudimentary version for intern task.
    
        Attributes:
        ----------
            STOPWORDS (lst) : List of Japanese stopwords
            NUMBERS (lst)   : List of digits as string type
            SPEC_CHARS (lst): List of special characters to remove
        
        Methods:
        --------
        __lowercase
        __exclude_stopwords
        __clean_numbers
        __reduce_whitespaces
        __strip_whitespaces
        __processing_function
        fit
        fit_transform
        transform            
    """
    # Initialise list of preprocessing methods to activate
    __preprocessing_stages = []
    
    # Attributes
    STOPWORDS = ['あそこ', 'あっ', 'あの', 'あのかた', 'あの人', 'あり', 'あります', 'ある', 'あれ', 'い', 'いう', 'います', 'いる', 'う', 'うち', 'え', 'お', 'および', 'おり', 'おります', 'か', 'かつて', 'から', 'が', 'き', 'ここ', 'こちら', 'こと', 'この', 'これ', 'これら', 'さ', 'さらに', 'し', 'しかし', 'する', 'ず', 'せ', 'せる', 'そこ', 'そして', 'その', 'その他', 'その後', 'それ', 'それぞれ', 'それで', 'た', 'ただし', 'たち', 'ため', 'たり', 'だ', 'だっ', 'だれ', 'つ', 'て', 'で', 'でき', 'できる', 'です', 'では', 'でも', 'と', 'という', 'といった', 'とき', 'ところ', 'として', 'とともに', 'とも', 'と共に', 'どこ', 'どの', 'な', 'ない', 'なお', 'なかっ', 'ながら', 'なく', 'なっ', 'など', 'なに', 'なら', 'なり', 'なる', 'なん', 'に', 'において', 'における', 'について', 'にて', 'によって', 'により', 'による', 'に対して', 'に対する', 'に関する', 'の', 'ので', 'のみ', 'は', 'ば', 'へ', 'ほか', 'ほとんど', 'ほど', 'ます', 'また', 'または', 'まで', 'も', 'もの', 'ものの', 'や', 'よう', 'より', 'ら', 'られ', 'られる', 'れ', 'れる', 'を', 'ん', '何', '及び', '彼', '彼女', '我々', '特に', '私', '私達', '貴方', '貴方方']
    
    NUMBERS = [str(x) for x in range(0,10)]
    
    SPEC_CHARS = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
    
    # Initialisation - set preprocessing methods on or off
    def __init__(self, p_lowercase=True, p_exclude_stopwords=True, 
                 p_clean_numbers=True, p_strip_whitespaces=True,
                 p_reduce_whitespaces=True):
             
        if p_lowercase == True:
             self.__preprocessing_stages.append(self.__lowercase)
        if p_exclude_stopwords == True:
            self.__preprocessing_stages.append(self.__exclude_stopwords)
        if p_clean_numbers == True:
             self.__preprocessing_stages.append(self.__clean_numbers)   
        if p_strip_whitespaces == True:
             self.__preprocessing_stages.append(self.__strip_whitespaces)    
        if p_reduce_whitespaces == True:
             self.__preprocessing_stages.append(self.__reduce_whitespaces)  
    
    # Text processing methods
    def __lowercase(self, p_str_input):
        p_str_input = str(p_str_input)        
        return str.lower(p_str_input)
    
    def __exclude_stopwords(self, p_str_input):
        p_str_input = str(p_str_input)
        return " ".join([word for word in p_str_input.split() if word not in self.STOPWORDS])                                   
    
    def __clean_numbers(self, p_str_input):
        p_str_input = str(p_str_input)        
        for char in self.NUMBERS:
            p_str_input = p_str_input.replace(char, ' ')  
        return p_str_input    
    
    def __reduce_whitespaces(self, p_str_input):
        p_str_input = str(p_str_input)        
        return ' '.join(p_str_input.split())
                                
    def __strip_whitespaces(self, p_str_input):
        p_str_input = str(p_str_input)        
        return p_str_input.rstrip()
    
    # Method for processing the other methods
    def __processing_function(self, raw_documents, y=None):
        
        processed_documents = []
        
        for document in raw_documents:
            for preprocessing_stage in self.__preprocessing_stages:
                document = preprocessing_stage(p_str_input=document)
                
            processed_documents.append(document)
            
        return processed_documents
    
    # Methods required for sklearn compatibility
    def fit(self, raw_documents, y=None):
        return self.__processing_function(raw_documents=raw_documents)

    def fit_transform(self, raw_documents, y=None):
        return self.__processing_function(raw_documents=raw_documents)

    def transform(self, raw_documents, copy=True):
        return self.__processing_function(raw_documents=raw_documents)

### Build Machine Learning Pipeline

Split the data randomly into 80%-20% training-test. Make sure the sentiment distributions are the same in the training and test data.

In [6]:
# Your code and/or markup here.

Display sentiment distributions of training and test set in stacked bar charts.

In [ ]:
# Your code and/or markup here.

Build the machine learning pipeline by:
- Instantiating the preprocessor class and activating your chosen cleaning methods.
- Instantiating the TweetTokenizer with whatever parameters you see fit **(done for you)**.
- Instantiating the TF-IDF vectorizer with whatever parameters you see fit.
- Instantiating a classifier such as LinearSVC with whatever paramters you see fit. You can research sklearn and select any classifier you desire.
- Create a pipeline object which first preprocesses, then vectorises, then classifies.

In [4]:
# Instantiate tweet tokeniser object
tweet_tok = TweetTokenizer(strip_handles=True, reduce_len=True)

# Your code and/or markup here.

### Training & Evaluation

Train your model and determine it's performance using the standard sklearn classification report metrics.
- Fit the pipeline.
- Predict with the trained pipeline.
- Create the metrics report and display it.

In [5]:
# Your code and/or markup here.